In [389]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import random

In [390]:
## Neuron Class Object ##
class Neuron:
    def __init__(self, weight, bias):
        self.weight = weight
        self.bias = bias
        
    def feedforward(self, inputs, act_type):
        total = np.dot(self.weight, inputs) + self.bias
        if act_type == 0:
            return sigmoid(total)
        elif act_type == 1:
            return softmax(total)
        elif act_type == 2:
            return relu(total)

In [391]:
## Neural Network Class Object ##
class Neural_Net:
    def __init__(self, n_num, weight, bias):
        self.weight = weight
        self.bias = bias
        self.n_num = n_num
        cur_layer = []
        self.all_layer = []
        for j in range(len(n_num)-1):
            for i in range(n_num[j+1]):
                cur_layer.append(Neuron(weight[j][i], bias[j][i]))
            self.all_layer.append(cur_layer)
            cur_layer = []
            
    def feedforward(self, i_data):
        cur_output = []
        all_output = []
        all_output.append(i_data.tolist())
        for j in range(len(self.n_num)-1):
            for i in range(self.n_num[j+1]):
                x = self.all_layer[j][i].feedforward(all_output[j],0)
                cur_output.append(x)
            if type(cur_output)==list:
                all_output.append(cur_output)
            else:
                all_output.append(cur_output.tolist())
            cur_output = []
        return all_output
    
    def backpropagation(self, pred, i_y):
        dim = len(pred)
        deriv_last = deriv_loss(i_y, pred[dim-1])
        g_field=[]
        for j in range(1, dim-1):
            deriv_input = deriv_hidden(pred[j-1], self.weight[j-1], self.bias[j-1])
            deriv = deriv_input
            for i in range(j, dim-1):
                deriv = np.matmul(deriv_pred(pred[i], self.weight[i], self.bias[i]), deriv)
            deriv = np.matmul(deriv_last, deriv)
            g_field.append(deriv)
        g_field_last = np.matmul(deriv_last, deriv_hidden(pred[dim-2], self.weight[dim-2], self.bias[dim-2]))
        g_field.append(g_field_last)
        w_field = []
        b_field = []
        for i in range(len(g_field)):
            w_field.append(np.reshape(g_field[i][:len(g_field[i])-len(self.weight[i])], (self.n_num[i+1],self.n_num[i])).tolist())
            b_field.append(g_field[i][len(g_field[i])-len(self.weight[i]):].tolist())
        return w_field, b_field
    
    def gradient_descent(self, w_field, b_field, learning_rate):
        for i in range(0, 2):
            self.weight[i] = np.subtract(self.weight[i],np.multiply(learning_rate, w_field[i])).tolist()
            self.bias[i] = np.subtract(self.bias[i], np.multiply(learning_rate, b_field[i])).tolist()
        return self.weight, self.bias

In [392]:
## HELPER FUNCTIONS ##
def sigmoid(x):
    return 1/(1 + np.exp(-x))

def deriv_sigmoid(x):
    fx = sigmoid(x)
    return fx * (1 - fx)

def nth_deriv_sigmoid(x, n):
    coeff = np.zeros(n + 1, dtype=int)
    coeff[0] = 1
    for i in range(1, n + 1):
        for j in range(i, -1, -1):
            coeff[j] = -j * coeff[j - 1] + (j + 1) * coeff[j]
    deriv = 0.0
    for i in range(n, -1, -1):
        deriv = sigmoid(x) * (coeff[i] + res)
    return deriv

def deriv_relu(x):
    if x > 0:
        return 1
    else:
        return 0

def deriv_loss(y_true, y_pred):
    return -2*(y_true - y_pred)

def deriv_pred(hidden, o_weight, o_bias):
    return o_weight * deriv_sigmoid(np.dot(o_weight, hidden) + o_bias)[:,None]

def deriv_hidden(i_data, weight, bias):
    weight = np.array(weight)
    n_dim = weight.shape[0]
    w_dim = weight.shape[1]
    derv = np.zeros((n_dim, n_dim * w_dim + n_dim))
    for i in range(0,n_dim):
        derv[i : i+1, i*w_dim : (i+1)*w_dim] = np.array(i_data) * deriv_sigmoid(np.dot(weight[i,:],i_data) + bias[i])
        derv[i : i+1, n_dim*w_dim+i : n_dim*w_dim+i+1] = deriv_sigmoid(np.dot(weight[i,:],i_data) + bias[i])
    return derv

def mse_loss(y_true, y_pred):
    return ((y_true - y_pred) ** 2).mean()

In [520]:
## Input Data ##
import csv
with open('Gender.csv') as csvfile: ## Weight, Height, Body type Data from Kaggle 
## https://www.kaggle.com/yersever/500-person-gender-height-weight-bodymassindex ##

    #readCSV = csv.reader(csvfile, delimiter=',',quoting=csv.QUOTE_NONNUMERIC)
    readCSV = csv.reader(csvfile, delimiter=',')
    sex = []
    weight = []
    height = []
    index = []
    for row in readCSV:
        sex.append(row[0])
        height.append(row[1])
        weight.append(row[2])
        index.append(row[3])
        
sex = np.array(sex[1:])
index = np.array(index[1:]).astype(int)
#index = np.subtract(rindex,round(np.mean(rindex)))
rheight = np.array(height[1:]).astype(int)
height = np.subtract(rheight,round(np.mean(rheight)))
rweight = np.array(weight[1:]).astype(int)
weight = np.subtract(rweight,round(np.mean(rweight)))
sex = np.where(sex=='Male',0,1)
i_data = np.transpose(np.array([weight, height, index, sex])).astype(int)
#i_data = sorted(i_data, key=lambda x : x[0])
#print(i_data)
#bot_10 = round(0.10 * len(i_data))
#top_10 = round(0.90 * len(i_data))
#i_data = i_data[:top_10]
#i_data = i_data[bot_10:]
#i_data = sorted(i_data, key=lambda x : x[1])
#bot_10 = round(0.05 * len(i_data))
#top_10 = round(0.95 * len(i_data))
#i_data = i_data[:top_10]
#i_data = i_data[bot_10:]
np.random.shuffle(i_data)
print(i_data)
#for i in range(len(i_data)):
#    i_data[i] = i_data[i].tolist()
#batch = np.array_split(i_data,6);
#print(batch)
#print(batch[0].shape)
#i_data = np.array([[-2, -1, 1],[25, 6, 0],[17, 4, 0],[-15, -6, 1]])
#i_data = np.array([[-2, -1,1]])
#print(i_data.shape)
#n = i_data.shape[1]
#i_x = i_data[: , 0:n-1]
#i_y = i_data[: , n-1:n]
#print(i_x)
#print(i_y)

[[-48  -3   2   1]
 [-46 -20   3   1]
 [ 12  20   4   1]
 ...
 [ 49   7   5   1]
 [-40 -13   3   1]
 [-38  16   2   0]]


In [521]:
## RANDOMIZE WEIGHTS ##
def set_hyper(n_num, x_dim):
    n_num.insert(0, x_dim)
    weight = []
    bias = []
    for i in range(len(n_num)-1):
        weight.append(np.transpose(np.random.normal(size=(n_num[i], n_num[i+1]))).tolist())
        bias.append(np.random.normal(size=(n_num[i+1])).tolist())
    return n_num, weight, bias

In [529]:
# Training Function ##
def train(i_data, n_vec, learning_rate, epoch):
    n = i_data.shape[1]
    obs_n = i_data.shape[0]
    i_y = i_data[: , n-1:n]
    i_x = i_data[: , 0:n-1]
    
    n_num, weight, bias = set_hyper(n_vec, n-1)
    
    #weight = [[[0.5,0.4],[0.3,0.2]],[[0.4,0.3],[0.2,0.1]],[[0.6,0.7]]]
    #bias = [[0.5,0.4],[0.4,0.3],[0.3]]
    #print(weight)
    #print(bias)
    
    for j in range(epoch):
        loss = 0
        #np.random.shuffle(i_data)
        i_y = i_data[:, n-1:n]
        i_x = i_data[:, 0:n-1]
        for i in range(0,obs_n):
            #print("\n")
            #print("Loop Number:",i)
            network = Neural_Net(n_num, weight, bias)
            y_pred = network.feedforward(i_x[i, :])
            w_field,b_field = network.backpropagation(y_pred, i_y[i])
            weight, bias = network.gradient_descent(w_field, b_field, learning_rate)
            #print("True Y", i_y[i])
            #print("W_field", w_field)
            #print("B_field", b_field)
            #print("New Weight", weight)
            #print("New Bias", bias)
            #print("Predicted Y", y_pred)
            loss = loss + mse_loss(i_y[i], y_pred[2])
            #print("Loss:",loss)
        loss = loss / i_data.shape[0]
        print("Epoch %d loss: %.3f" % (j + 1, loss))
    return network

In [ ]:
network = train(i_data, [100,1,1], 0.001, 1000)

Epoch 1 loss: 0.427
Epoch 2 loss: 0.420
Epoch 3 loss: 0.414
Epoch 4 loss: 0.407
Epoch 5 loss: 0.402
Epoch 6 loss: 0.398
Epoch 7 loss: 0.397
Epoch 8 loss: 0.397
Epoch 9 loss: 0.398
Epoch 10 loss: 0.399
Epoch 11 loss: 0.401
Epoch 12 loss: 0.402
Epoch 13 loss: 0.403
Epoch 14 loss: 0.404
Epoch 15 loss: 0.405
Epoch 16 loss: 0.406
Epoch 17 loss: 0.406
Epoch 18 loss: 0.406
Epoch 19 loss: 0.406
Epoch 20 loss: 0.406
Epoch 21 loss: 0.406
Epoch 22 loss: 0.406
Epoch 23 loss: 0.405
Epoch 24 loss: 0.405
Epoch 25 loss: 0.404
Epoch 26 loss: 0.404
Epoch 27 loss: 0.403
Epoch 28 loss: 0.402
Epoch 29 loss: 0.401
Epoch 30 loss: 0.399
Epoch 31 loss: 0.398
Epoch 32 loss: 0.395
Epoch 33 loss: 0.393
Epoch 34 loss: 0.390
Epoch 35 loss: 0.387
Epoch 36 loss: 0.383
Epoch 37 loss: 0.380
Epoch 38 loss: 0.377
Epoch 39 loss: 0.374
Epoch 40 loss: 0.371
Epoch 41 loss: 0.368
Epoch 42 loss: 0.365
Epoch 43 loss: 0.362
Epoch 44 loss: 0.360
Epoch 45 loss: 0.359
Epoch 46 loss: 0.359
Epoch 47 loss: 0.361
Epoch 48 loss: 0.362
E

In [512]:
## Average Metrics ##
print("Height Average(CM)", round(np.mean(rheight)))
print("Weight Average(KG)", round(np.mean(rweight)))
print("Body Type Index Average", round(np.mean(rindex)))

Height Average(CM) 170.0
Weight Average(KG) 106.0
Body Type Index Average 4.0


In [513]:
## Prediction ##
## [Weight, Height, Body Type] ##
Boy = np.array([-26,20,3]) # Kevin 180 pounds, 74 inches
Girl = np.array([-42, 11,0])  # Alina 141 pounds, 71 inches
p1 = network.feedforward(Boy)
p2 = network.feedforward(Girl)
print("Kevin: %.3f",  p1[2])
print("Alina: %.3f", p2[2])

Kevin: %.3f [0.42213294983886634]
Alina: %.3f [0.5177929978865558]
